In [1]:
import numpy as np
np.set_printoptions(linewidth=np.inf)

In [64]:
def propagator(layout, initial_coord):
    def update_pos(pos, direc):
        delta = {0: (0,1), 1: (1,0), 2: (0,-1), 3: (-1,0)} #rightward, downward, leftward, upward, resp
        pos = tuple(map(sum,zip(pos,delta[direc])))
        if pos[0] not in range(len(layout)) or pos[1] not in range(len(layout[0])):
            return None
        energized[pos[0]][pos[1]] = True
        new_beams.append((pos, direc))
        return None

    energized = [[False for _ in range(len(layout[0]))] for _ in range(len(layout))]
    beams = [initial_coord]
    initial_pos, direc = initial_coord
    energized[initial_pos[0]][initial_pos[1]] = True
    record = set([initial_coord])

    while beams != []:
        new_beams = []
        for coord in beams:
            pos, direc = coord
            if pos in record:
                continue            
            current_tile = layout[pos[0]][pos[1]]
            if current_tile=="." or (current_tile=="-" and direc in [0,2]) or (current_tile=="|" and direc in [1,3]):
                update_pos(pos, direc)
            if current_tile == "/":
                update_pos(pos, 3 - direc) #dir: 0<->3, 1<->2
            if current_tile == "\\":
                update_pos(pos, direc - direc%2 + (direc+1)%2 ) #dir: 0<->1, 2<->3 
            if current_tile=="-" and direc in [1,3]:
                update_pos(pos, 0), update_pos(pos, 2)
                record.add(pos) #recording only at the splits suffices (though checking always)
            if current_tile=="|" and direc in [0,2]:
                update_pos(pos, 1), update_pos(pos, 3)
                record.add(pos) #recording only at the splits suffices (though checking always)
        beams = new_beams
    return sum([el for row in energized for el in row])

with open("input16.dat", "r") as f:
    layout = f.readlines()
layout = [line.removesuffix("\n") for line in layout]

initial_coord = ((0,0),0)
propagator(layout, initial_coord)

6514

In [65]:
with open("input16.dat", "r") as f:
    layout = f.readlines()
layout = [line.removesuffix("\n") for line in layout]

edge_coord = [((i,0),0) for i in range(len(layout))] + [((i,-1),2) for i in range(len(layout))] + [((0,j),1) for j in range(len(layout[0]))] + [((-1,j),3) for j in range(len(layout[0]))]
max_energ = 0
for initial_coord in edge_coord:
    n_energ = propagator(layout, initial_coord)
    if max_energ < n_energ:
        max_energ = n_energ
max_energ

8089

From this it's clear that NumPy is much slower for these problem sizes: with string edition, it took 5s to solve Part 2, with NumPy, around 50s (sometimes more)

The two variants below, saving every position+direction, or just at the splitters, have no real difference in performance

In [66]:
def propagator(layout, initial_pos):
    def update_pos(pos, dir):
        delta = {0:np.array([0,1]), 1:np.array([1,0]), 2:np.array([0,-1]), 3:np.array([-1,0])} #rightward, downward, leftward, upward, resp
        pos = pos + delta[dir]
        if pos[0] not in range(layout.shape[0]) or pos[1] not in range(layout.shape[1]) or tuple(pos)+(dir,) in record:
            return None
        energized[tuple(pos)] = True
        new_beams.append(np.array(np.concatenate((pos,np.array([dir])))))
        record.add(tuple(pos)+(dir,))
        return None
    
    energized = np.zeros(layout.shape, bool)
    beams = [initial_pos]
    energized[tuple(initial_pos[0:2])] = True
    record = set([tuple(beams[0])]).copy()

    while beams != []:
        new_beams = []
        for coord in beams:
            pos, direc = coord[0:2], coord[2]
            current_tile = layout[tuple(pos)]
            if current_tile=="." or (current_tile=="-" and direc in [0,2]) or (current_tile=="|" and direc in [1,3]):
                update_pos(pos, direc)
            if current_tile == "/":
                update_pos(pos, 3 - direc) #dir: 0<->3, 1<->2
            if current_tile == "\\":
                update_pos(pos, direc - direc%2 + (direc+1)%2 ) #dir: 0<->1, 2<->3 
            if current_tile=="-" and direc in [1,3]:
                update_pos(pos, 0), update_pos(pos, 2)
            if current_tile=="|" and direc in [0,2]:
                update_pos(pos, 1), update_pos(pos, 3)
        beams = new_beams
    return np.count_nonzero(energized)

layout = np.genfromtxt("input16.dat", str, delimiter=1, comments=None)
initial_pos = np.array([0,0,0])

propagator(layout, initial_pos)

6514

In [62]:
def propagator(layout, initial_pos):
    def update_pos(pos, dir):
        delta = {0:np.array([0,1]), 1:np.array([1,0]), 2:np.array([0,-1]), 3:np.array([-1,0])} #rightward, downward, leftward, upward, resp
        pos = pos + delta[dir]
        if pos[0] not in range(layout.shape[0]) or pos[1] not in range(layout.shape[1]):
            return None
        energized[tuple(pos)] = True
        new_beams.append(np.array(np.concatenate((pos,np.array([dir])))))        
        return None
    
    energized = np.zeros(layout.shape, bool)
    beams = [initial_pos]
    energized[tuple(initial_pos[0:2])] = True
    record = set()

    while beams != []:
        new_beams = []
        for coord in beams:
            pos, direc = coord[0:2], coord[2]
            if tuple(pos) in record:
                continue
            current_tile = layout[tuple(pos)]
            if current_tile=="." or (current_tile=="-" and direc in [0,2]) or (current_tile=="|" and direc in [1,3]):
                update_pos(pos, direc)
            if current_tile == "/":
                update_pos(pos, 3 - direc) #dir: 0<->3, 1<->2
            if current_tile == "\\":
                update_pos(pos, direc - direc%2 + (direc+1)%2 ) #dir: 0<->1, 2<->3 
            if current_tile=="-" and direc in [1,3]:
                update_pos(pos, 0), update_pos(pos, 2)
                record.add(tuple(pos))
            if current_tile=="|" and direc in [0,2]:
                update_pos(pos, 1), update_pos(pos, 3)
                record.add(tuple(pos))
        beams = new_beams
    return np.count_nonzero(energized)

layout = np.genfromtxt("input16.dat", str, delimiter=1, comments=None)
initial_pos = np.array([0,0,0])

propagator(layout, initial_pos)

6514

In [67]:
layout = np.genfromtxt("input16.dat", str, delimiter=1, comments=None)
edge_pos = [np.array([i,0,0]) for i in range(layout.shape[0])] + [np.array([i,-1,2]) for i in range(layout.shape[0])] + [np.array([0,j,1]) for j in range(layout.shape[1])] + [np.array([-1,j,3]) for j in range(layout.shape[1])]
max_energ = 0
for initial_pos in edge_pos:
    n_energ = propagator(layout, initial_pos)
    if max_energ < n_energ:
        max_energ = n_energ
max_energ

8089